In [1]:
import numpy as np
import pandas as pd
import math
import os
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

In [2]:
user_subtag = pd.read_csv('/home/jovyan/df-smart-channel/graph/data/preprocessed/user_subtag.csv')

In [3]:
context = pd.read_csv('/home/jovyan/df-smart-channel/graph/data/preprocessed/context_1217_0131.csv')

In [4]:
user_subtag.head()

,cust_no,date,mobile_login_90,dd_my,dd_md,onlymd_ind,onlycc_ind,efingo_card_ind,cl_cpa_amt,fc_ind
0,++zmpPGNtNirTv2mCQb9mQ==,20211226,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0
1,+2/rB1yYcMFVRjbETKUlkg==,20211226,1.0,0.0,0.0,0.0,1.0,0.0,NaN,0
2,+2csF6o9VzkGxSFb87i3UQ==,20211226,1.0,0.0,1.0,0.0,0.0,0.0,NaN,0
3,+4f4gQoJE9IF4i+hUbg5XA==,20211226,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4,+5+nzD+wwki5IB/TVpaFZw==,20211226,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
# le = LabelEncoder()
# context.cust_no = le.fit_transform(context.cust_no)

In [6]:
# user_subtag.cust_no = le.transform(user_subtag.cust_no)

In [7]:
user_subtag['is_in_context'] = user_subtag.cust_no.isin(context.cust_no.unique())

In [8]:
user_subtag.shape

(874292, 11)

In [9]:
user_subtag = user_subtag[user_subtag.is_in_context]

In [10]:
# train test
train_date = 20211215
test_date = 20211226

In [13]:
user_subtag.date.unique()

array([20211226])

In [12]:
user_subtag[user_subtag.date == 20211225]

,cust_no,date,mobile_login_90,dd_my,dd_md,onlymd_ind,onlycc_ind,efingo_card_ind,cl_cpa_amt,fc_ind,is_in_context


In [10]:
user_subtag = user_subtag.reset_index()

In [11]:
user_subtag.head()

,index,cust_no,mobile_login_90,dd_my,dd_md,onlymd_ind,onlycc_ind,efingo_card_ind,cl_cpa_amt,fc_ind,is_in_context
0,0,++zmpPGNtNirTv2mCQb9mQ==,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0,True
1,3,+2csF6o9VzkGxSFb87i3UQ==,1.0,0.0,1.0,0.0,0.0,0.0,NaN,0,True
2,4,+4f4gQoJE9IF4i+hUbg5XA==,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0,True
3,6,+6TjjYio97pnzZ3tr1zSBw==,1.0,0.0,0.0,0.0,0.0,0.0,500000.0,0,True
4,7,+6sDXXep82BZEI6tTPZEEg==,1.0,0.0,0.0,0.0,1.0,0.0,NaN,0,True


In [12]:
le = LabelEncoder()
context.cust_no = le.fit_transform(context.cust_no)

In [13]:
user_subtag.cust_no = le.transform(user_subtag.cust_no)

In [14]:
# 行銀活躍用戶
active_acct = user_subtag.mobile_login_90
# 有外幣帳戶
w_fx_acct = user_subtag.dd_my
# 存戶
w_digital_acct = user_subtag.dd_md
# 純存戶
onlymd_acct = user_subtag.onlymd_ind
# 純卡戶
onlycc_acct = user_subtag.onlycc_ind
# 卡存戶有e.Fingo指定卡
w_efingo_card = user_subtag.efingo_card_ind
# 有信貸者
w_debit = user_subtag.cl_cpa_amt.notna()
# 無理專顧客
wo_fc = user_subtag.fc_ind == 0

In [15]:
res_df = {}
for i, ids in enumerate(user_subtag.cust_no):
    subtags_list = []
    if active_acct[i]: subtags_list.append(0);
    if w_fx_acct[i]: subtags_list.append(1);
    if w_digital_acct[i]: subtags_list.append(2);
    if onlymd_acct[i]: subtags_list.append(3);        
    if onlycc_acct[i]: subtags_list.append(4);
    if w_efingo_card[i]: subtags_list.append(5);
    if w_debit[i]: subtags_list.append(6);
    if wo_fc[i]: subtags_list.append(7);
    res_df[ids] = subtags_list

In [14]:
res_df

NameError: name 'res_df' is not defined

In [17]:
test_date = 20211226

In [19]:
export_file(date = str(test_date), pairs_dict_train = res_df, pairs_dict_test = res_df )

write to file path: <_io.TextIOWrapper name='date=20211226/train_user_subtag.txt' mode='w' encoding='UTF-8'>
576691
write to file path: <_io.TextIOWrapper name='date=20211226/test_user_subtag.txt' mode='w' encoding='UTF-8'>
576691


In [18]:
def export_file(date, pairs_dict_train, pairs_dict_test):
    dir_name = f'date={date}'
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
        
    ## train
    file_path = os.path.join(dir_name, 'train_user_subtag.txt')
    file = open(file_path,"w")
    print(f'write to file path: {file}')
    counts = 0
    for key,value in pairs_dict_train.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                else:
                    file.write(' %s' % int(item))
        file.write('\n')
        counts += 1
    print(counts)
    file.close()
    ## test
    file_path = os.path.join(dir_name, 'test_user_subtag.txt')
    file = open(file_path,"w")
    counts = 0
    print(f'write to file path: {file}')
    
    for key,value in pairs_dict_test.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                    file.write('\n')
                else:
                    file.write(' %s' % int(item))
        counts += 1
    print(counts)
    file.close()

In [2]:
# context = pd.read_csv('..//data/preprocessed//context_1217_1230.csv')
context = pd.read_csv('/home/jovyan/df-smart-channel/graph/data/preprocessed/context_1217_0131.csv')

In [3]:
le = LabelEncoder()
context.cust_no = le.fit_transform(context.cust_no)
context.item_id = le.fit_transform(context.item_id)

In [4]:
user_list = context.cust_no.unique()
item_list = context.item_id.unique()

In [5]:
len(user_list)

576702

In [7]:
context['cust_no'].unique()

array([    53,     55,     93, ..., 451199, 477546, 489807])

In [4]:
# user_list = context.cust_no.unique()
# item_list = context.item_id.unique()